In [ ]:
import pandas as pd
from scipy.stats import ttest_ind

menu_a = pd.read_csv('data/MENU_B_STATS.csv')
menu_a = menu_a.set_index("ID")
demographics = pd.read_csv(f'data/demographics.csv')
demographics = demographics.rename(columns={"Participant ID": "ID"})
demographics = demographics.set_index("ID")

menu_a = menu_a.join(demographics)
menu_male = menu_a[menu_a["Gender"] == 1]
menu_female = menu_a[menu_a["Gender"] == 2]

In [ ]:
print("Warning")
stat, p = ttest_ind(menu_male["Warning Dwell Time"], menu_female["Warning Dwell Time"], equal_var=False, alternative="two-sided")
print(f'stat={stat:.3f}, p={p:.3f}')
print(f'{"Significant" if p < 0.05 else "Not significant"}, {"***" if p < 0.001 else "**" if p < 0.01 else "*" if p < 0.05 else ""}')

stat, p = ttest_ind(menu_male["Warning Fixations"], menu_female["Warning Fixations"], equal_var=False, alternative="greater")
print(f'stat={stat:.3f}, p={p:.3f}')
print(f'{"Significant" if p < 0.05 else "Not significant"}, {"***" if p < 0.001 else "**" if p < 0.01 else "*" if p < 0.05 else ""}')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import imageio
import cv2
from matplotlib import colors

def fixation_detection(data, max_dist=25, min_dur=50):
	s_fix = []
	e_fix = []

	x = data["x"].to_numpy()
	y = data["y"].to_numpy()
	time = data["VideoTime"].to_numpy()

	si = 0
	fix_start = False
	for i in range(1,len(x)):
		squared_distance = ((x[si]-x[i])**2 + (y[si]-y[i])**2)
		dist = 0.0
		if squared_distance > 0:
			dist = squared_distance**0.5
		if dist <= max_dist and not fix_start:
			si = 0 + i
			fix_start = True
			s_fix.append([time[i]])
		elif dist > max_dist and fix_start:
			fix_start = False
			if time[i-1]-s_fix[-1][0] >= min_dur:
				e_fix.append([s_fix[-1][0], time[i-1], time[i-1]-s_fix[-1][0], x[si], y[si]])
			else:
				s_fix.pop(-1)
			si = 0 + i
		elif not fix_start:
			si += 1
	if len(s_fix) > len(e_fix):
		e_fix.append([s_fix[-1][0], time[len(x)-1], time[len(x)-1]-s_fix[-1][0], x[si], y[si]])
	return s_fix, e_fix

overlay_width = 7680//6
overlay_height = 3840//6

canvas_x_1 = (overlay_width*1.35)//3
canvas_x_2 = (overlay_width*1.75)//3

canvas_y_1 = (overlay_height*1.2)//4
canvas_y_2 = (overlay_height*2.4)//4

overlay_img = imageio.imread(f"data/screenshot_B.PNG")
overlay_img = cv2.resize(overlay_img, (overlay_width, overlay_height))

m_fixations_x = []
m_fixations_y = []
f_fixations_x = []
f_fixations_y = []

demographics = pd.read_csv(f'data/demographics.csv')
for p_id in demographics["Participant ID"]:
	p_gender = demographics[demographics["Participant ID"] == p_id]["Gender"].item()
	p_group = demographics[demographics["Participant ID"] == p_id]["Group"].item()
	if p_group == 1:
		continue
	plot_color = "r" if p_gender == 1 else "g"

	tracking_data = pd.read_csv(f'data/tracking/{p_id}_MENU_B.csv')
	tracking_data['x'] = tracking_data['gaze coord phi']/np.pi
	tracking_data['y'] = tracking_data[" gaze coord theta"]/(np.pi/2)
	tracking_data['x'] = (tracking_data['x']+1)*0.5*overlay_width
	tracking_data['y'] = (tracking_data['y']+1)*0.5*overlay_height
	tracking_data = tracking_data[tracking_data["VideoTime"] > 95]
	tracking_data = tracking_data[tracking_data["VideoTime"] < 215]

	s_fix, e_fix = fixation_detection(tracking_data, max_dist=15, min_dur=0.05)

	tracking_fixations = pd.DataFrame({"VideoTime": np.array(e_fix)[:,0], "x": np.array(e_fix)[:,3], "y": np.array(e_fix)[:,4]})
	tracking_fixations = tracking_fixations[tracking_fixations["VideoTime"] > 95]
	tracking_fixations = tracking_fixations[tracking_fixations["VideoTime"] < 215]

	if p_gender == 1:
		m_fixations_x.append(tracking_fixations['x'])
		m_fixations_y.append(tracking_fixations['y'])
	else:
		f_fixations_x.append(tracking_fixations['x'])
		f_fixations_y.append(tracking_fixations['y'])

	# plt.figure(figsize=(8, 8))
	# plt.imshow(np.flip(overlay_img, axis=0), origin='lower')
	# plt.hist2d(tracking_fixations['x'], tracking_fixations['y'],
	# 		   cmap="YlOrRd" if p_gender == 1 else "YlGnBu",
	# 		   norm=colors.LogNorm(),
	# 		   range=[[canvas_x_1, canvas_x_2], [canvas_y_1, canvas_y_2]],
	# 		   bins=[20, 20], alpha=0.5)
	# plt.xlim(canvas_x_1, canvas_x_2)
	# plt.ylim(canvas_y_1, canvas_y_2)
	# plt.savefig(f"heatmaps/{'M' if p_gender == 1 else 'F'}_{p_id}.png", bbox_inches='tight')
	# plt.close()

In [ ]:
# m_fixations_x = np.hstack(m_fixations_x)
# m_fixations_y = np.hstack(m_fixations_y)
# f_fixations_x = np.hstack(f_fixations_x)
# f_fixations_y = np.hstack(f_fixations_y)

plt.figure(figsize=(8, 8))
plt.imshow(np.flip(overlay_img, axis=0), origin='lower')
plt.hist2d(m_fixations_x, m_fixations_y,
		   cmap="YlOrRd",
		   norm=colors.LogNorm(),
		   range=[[canvas_x_1, canvas_x_2], [canvas_y_1, canvas_y_2]],
		   bins=[20, 20], alpha=0.5)
plt.xlim(canvas_x_1, canvas_x_2)
plt.ylim(canvas_y_1, canvas_y_2)
plt.savefig(f"heatmaps/M_aggregate_B.png", bbox_inches='tight')
plt.close()